In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
#converting categorical to numerical value
dataset=pd.get_dummies(dataset,dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf','sigmoid'],
    'gamma': ['scale', 'auto']
}


# Store the best model under grid parameter; f1_weighted can be suited for imbalanced data
grid = GridSearchCV(SVC(probability=True, random_state=0), param_grid, 
                    refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fitting the model for grid search
grid.fit(X_train, y_train)
        

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=SVC(probability=True, random_state=0), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto'],
                         'kernel': ['rbf', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [13]:
# Store cross-validation results
cv_results = grid.cv_results_
# Uncomment to print all cross-validation results
# print(cv_results)

# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test)

# Using the final best fit model to test X_test
grid_predictions = grid.predict(X_test)
        

In [14]:
# Compute and print the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions )      

In [15]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86        85
           1       0.89      0.51      0.65        49

    accuracy                           0.80       134
   macro avg       0.83      0.74      0.75       134
weighted avg       0.82      0.80      0.78       134



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.8129651860744298)

In [18]:
print(cm)

[[82  3]
 [24 25]]


In [19]:
grid.predict([[40,300,1,]])

C:\Anaconda\envs\aiml\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([0])

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.046767,0.008905,0.015210,0.008077,0.1,scale,rbf,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}",0.776290,0.717410,0.621589,0.761537,0.721829,0.719731,0.054016,4
1,0.038787,0.011234,0.014444,0.002913,0.1,scale,sigmoid,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.509779,0.525300,0.501410,0.501410,0.501410,0.507862,0.009302,10
2,0.075629,0.016908,0.099643,0.170776,0.1,auto,rbf,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}",0.509779,0.525300,0.501410,0.501410,0.501410,0.507862,0.009302,10
3,0.044033,0.039842,0.043928,0.079962,0.1,auto,sigmoid,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.509779,0.525300,0.501410,0.501410,0.501410,0.507862,0.009302,10
4,0.035955,0.013446,0.018177,0.010973,1.0,scale,rbf,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.776290,0.717410,0.621589,0.783837,0.747667,0.729359,0.058759,3
5,0.030508,0.005844,0.015458,0.006774,1.0,scale,sigmoid,"{'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.442638,0.477832,0.460222,0.483331,0.480346,0.468874,0.015389,16
6,0.092759,0.026197,0.026911,0.013155,1.0,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.550758,0.605441,0.483019,0.501410,0.543089,0.536743,0.042640,9
7,0.035273,0.010264,0.018521,0.006210,1.0,auto,sigmoid,"{'C': 1, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.509779,0.525300,0.501410,0.501410,0.501410,0.507862,0.009302,10
8,0.044913,0.005065,0.014522,0.002255,10.0,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.782557,0.744194,0.651089,0.805463,0.747667,0.746194,0.052700,2
9,0.028261,0.010215,0.015304,0.008875,10.0,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.686940,0.664320,0.682050,0.775815,0.480346,0.657894,0.096847,5
